# Questions and Answers using Llama 2

Model: Llama 2 7b-chat

Data: Handbook test data set

In [6]:
from reginald.models.setup_llm import setup_llm

from reginald.models.models.llama_index import (
    setup_settings,
    LlamaIndexLlamaCPP,
    set_global_tokenizer,
    compute_default_chunk_size,
)


In [7]:
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response
from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama

from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)

from transformers import AutoTokenizer
import nest_asyncio

nest_asyncio.apply()

In [8]:
# Use ollama
ollama_llm = Ollama(model="llama2:7b-chat", request_timeout=60.0)

In [9]:
chunk_size = compute_default_chunk_size(
    max_input_size=4096, k=3
)  # calculate chunk size

In [10]:
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf"
).encode  # load tokenizer
set_global_tokenizer(tokenizer)

In [11]:
settings = setup_settings(
    llm=ollama_llm,
    max_input_size=4096,
    num_output=512,
    chunk_overlap_ratio=0.1,
    chunk_size=chunk_size,
    k=3,
    tokenizer=tokenizer,
)  # these are settings for the storage context

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Load in the Data

In [12]:
# To load in the handbook

storage_context = StorageContext.from_defaults(
    persist_dir="../../data/llama_index_indices/handbook/"
)

vector_index = load_index_from_storage(
    storage_context=storage_context,
    settings=settings,
)  # load the data index from storage

In [13]:
# To load in the example Paul Graham data
# reader = SimpleDirectoryReader("./data/paul_graham/")
# documents = reader.load_data()
# vector_index = VectorStoreIndex.from_documents(documents)

In [14]:
dataset_generator = DatasetGenerator.from_documents(
    vector_index.docstore.docs.values(),
    num_questions_per_chunk=1,
    show_progress=True,
    llm=ollama_llm
)

Parsing nodes:   0%|          | 0/39 [00:00<?, ?it/s]

/Users/kgoldmann/Documents/Projects/reginald/.venv/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:213: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


## Generate Questions

In [15]:
eval_questions = dataset_generator.generate_questions_from_nodes(
    5
)  # generate questions from the documents

100%|██████████| 5/5 [00:37<00:00,  7.55s/it]
/Users/kgoldmann/Documents/Projects/reginald/.venv/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:310: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [16]:
eval_questions

['Of course! Here are 5 questions based on the provided context information:',
 'What is the purpose of the "Contributing" section in the REG Handbook?',
 'Who is the intended audience for the guide in the "Contributing" section?',
 'How does the guide aim to make contributing to the handbook user-friendly?',
 'What is the main focus of the guide in terms of providing links for further reading?']

## Answer the Questions

In [17]:
evaluator_gpt4 = RelevancyEvaluator(llm=ollama_llm)

In [18]:
# for every value in eval_questions, get the response from the llm
all_dfs = []

for i in range(len(eval_questions)):
    query_engine = vector_index.as_query_engine()
    response_vector = query_engine.query(eval_questions[i])
    eval_result = evaluator_gpt4.evaluate_response(
        query=eval_questions[i], response=response_vector
    )

    # append to the all_dfs list
    all_dfs.append({'eval_result': eval_result,
                    'question': eval_questions[i],
                    'response': response_vector})


In [21]:
# extract the different elements
questions = [x['question'] for x in all_dfs]
response = [x['response'] for x in all_dfs]
sources = [x['response'].source_nodes for x in all_dfs]
#sources = [x[0].node.get_content() for x in sources]

# for the each sources, get the content as one string deliminated by ;
sources = [";".join([x.get_content() for x in y]) for y in sources]

match = [x['eval_result'].passing for x in all_dfs] # does the response match the source

In [25]:
# extract the questions
combined_df = pd.DataFrame({'questions': questions, 'response': response, 'sources': sources, 'match': match})
combined_df.to_csv("data/handbook_QandA.csv", index=False)

In [26]:
combined_df_out = combined_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        }
    )
display(combined_df_out)

## Ask Reginal the Questions

In [27]:
response_model = setup_llm(
    model="llama-index-llama-cpp",
    model_name="../../../llama-2-7b-chat.Q4_K_M.gguf",
    data_dir="../../data/",
    which_index="handbook",
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/kgoldmann/Library/Caches/llama_index/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:            

In [34]:
def get_response(question):
    resp = response_model.direct_message(message=que, user_id="")
    return(resp.message)

In [35]:
all_questions = combined_df['questions'].tolist()
reg_responses = []
for que in all_questions:
        reg_responses.append(get_response(que))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   14150.19 ms
llama_print_timings:      sample time =      17.09 ms /   139 runs   (    0.12 ms per token,  8131.51 tokens per second)
llama_print_timings: prompt eval time =   12778.71 ms /   887 tokens (   14.41 ms per token,    69.41 tokens per second)
llama_print_timings:        eval time =   10230.16 ms /   138 runs   (   74.13 ms per token,    13.49 tokens per second)
llama_print_timings:       total time =   23306.57 ms /  1025 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   14150.19 ms
llama_print_timings:      sample time =      27.86 ms /   242 runs   (    0.12 ms per token,  8687.22 tokens per second)
llama_print_timings: prompt eval time =   18033.65 ms /  1264 tokens (   14.27 ms per token,    70.09 tokens per second)
llama_print_timings:        eval time =   18838.52 ms /   241 runs   (   78.17 ms per token,    12.79 tokens per second)
llama_print_timings:       to

In [37]:
for i in reg_responses:
    print(i)
    print("\n")

  Great! Here are five questions based on the provided context information:
1. What is the main purpose of the "Contributing" section in the REG Handbook?
2. How does the section help create an inclusive and welcoming environment for new starters at REG?
3. Can you explain the significance of the "Buddy System" mentioned in the section?
4. What are some of the key technologies or tools mentioned in the section that new employees may need to know about?
5. How does the section encourage new employees to explore their interests and expand their skill sets within the context of their work at REG?


I read the following documents to compose this answer:
https://alan-turing-institute.github.io/REG-handbook/docs/regular_events/reading_groups/ (similarity: 0.21)

https://alan-turing-institute.github.io/REG-handbook/docs/how_we_work/ (similarity: 0.2)

https://alan-turing-institute.github.io/REG-handbook/docs/regular_events/ (similarity: 0.17)


  The purpose of the "Contributing" section in t

In [38]:
combined_df['reginald_response'] = reg_responses

In [39]:
combined_df

,questions,response,sources,match,reginald_response
0,Of course! Here are 5 questions based on the p...,"Of course, I'm here to help! Based on the prov...",Reading Groups As part of projects or for gene...,True,Great! Here are five questions based on the ...
1,"What is the purpose of the ""Contributing"" sect...","The purpose of the ""Contributing"" section in t...",Projects This section describes how we coordin...,True,"The purpose of the ""Contributing"" section in..."
2,Who is the intended audience for the guide in ...,"Based on the context provided, it appears that...",Discussions and Issues Contributing does not o...,True,"The intended audience for the ""Contributing""..."
3,How does the guide aim to make contributing to...,The guide aims to make contributing to the han...,Discussions and Issues Contributing does not o...,True,"The ""Contributing"" section of the REG (Resea..."
4,What is the main focus of the guide in terms o...,"Based on the provided context information, it ...",Discussions and Issues Contributing does not o...,True,"The ""Contributing"" section of the REG (Resea..."


In [64]:
# turing the response_vector into an object with attribute response
class Response:
    def __init__(self, response):
        self.response = response
        self.get_content = response
        self.source_nodes = response

response_objects = [Response(x) for x in reg_responses]
response_objects[0]

In [66]:
response_objects[0].get_content

'  Great! Here are five questions based on the provided context information:\n1. What is the main purpose of the "Contributing" section in the REG Handbook?\n2. How does the section help create an inclusive and welcoming environment for new starters at REG?\n3. Can you explain the significance of the "Buddy System" mentioned in the section?\n4. What are some of the key technologies or tools mentioned in the section that new employees may need to know about?\n5. How does the section encourage new employees to explore their interests and expand their skill sets within the context of their work at REG?\n\n\nI read the following documents to compose this answer:\nhttps://alan-turing-institute.github.io/REG-handbook/docs/regular_events/reading_groups/ (similarity: 0.21)\n\nhttps://alan-turing-institute.github.io/REG-handbook/docs/how_we_work/ (similarity: 0.2)\n\nhttps://alan-turing-institute.github.io/REG-handbook/docs/regular_events/ (similarity: 0.17)'

In [69]:
reg_dfs = []

for i in range(len(eval_questions)):
    query_engine = vector_index.as_query_engine()
    response_vector = query_engine.query(eval_questions[i])
    eval_result = .evaluate_response(
        query=eval_questions[i], response=response_vector
    )

# for i in range(len(all_questions)):
#     query_engine = vector_index.as_query_engine()
#     response_vector = reg_responses[i]
#     eval_result = evaluator_gpt4.evaluate_response(
#         query=all_questions[i], response=response_objects[i]
#     )

#     # append to the all_dfs list
#     reg_dfs.append({'eval_result': eval_result,
#                     'question': all_questions[i],
#                     'response': response_vector})

ValidationError: 1 validation error for EvaluationResult
contexts
  value is not a valid sequence (type=type_error.sequence)